# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.automl import AutoMLConfig
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from TrainCovid19Infections import clean_data
import os
import shutil

## Initialize Workspace

Create a workspace, if it doesn't exist, using the AzureML SDK

In [2]:
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: OptimizePipeline
Azure region: eastus2
Subscription id: c04b3d3f-4994-454d-96ff-aa3f2050b57f
Resource group: testingMLFunctionnalities


## Cluster

Get cluster if it exists else create one

In [3]:
amlcompute_cluster_name = "Covid19Cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
    print(compute_target.get_status().serialize())
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-20T15:13:39.626000+00:00', 'errors': None, 'creationTime': '2021-03-20T15:13:34.812815+00:00', 'modifiedTime': '2021-03-20T15:13:52.240694+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19InfectionsDataset"
description_text = "Covid19 Vaccination DataSet from Github"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./github/owid-covid-data.csv'],
                       target_path ='train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    original_path = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
    ds = TabularDatasetFactory.from_delimited_files(original_path, infer_column_types=True, separator=',', header=True)
    
    
    #ds = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
    
    #Register Dataset in Workspace
    dataset = ds.register(workspace=ws,
                          name=key,
                          description=description_text)

df = dataset.to_pandas_dataframe().fillna(0)
df.describe()

Uploading an estimated of 1 files
Uploading ./github/owid-covid-data.csv
Uploaded ./github/owid-covid-data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
count,7.600800e+04,76008.000000,76008.000000,7.600800e+04,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,...,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000
mean,6.610629e+05,5121.867198,5056.453239,1.700790e+04,114.355726,113.112097,7828.250439,66.352370,65.222400,160.007469,...,27.703192,7.881587,5.024407,17414.585578,236.143373,7.253355,23.491059,2.548535,69.503854,0.667584
std,4.676108e+06,32165.477705,31613.208461,1.076847e+05,674.708492,652.071812,15579.320551,168.173557,140.944476,314.049383,...,12.417608,6.483054,4.370750,19610.341599,133.720476,4.293250,33.330064,2.519366,17.536964,0.246658
min,0.000000e+00,-74347.000000,-6223.000000,0.000000e+00,-1918.000000,-232.143000,0.000000,-2153.437000,-276.825000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.710000e+02,1.000000,4.429000,1.000000e+01,0.000000,0.000000,122.698250,0.061000,0.739000,1.589750,...,19.600000,3.008000,1.783000,2896.913000,140.448000,4.610000,0.000000,0.700000,66.470000,0.555000
50%,7.783000e+03,55.000000,62.714000,1.290000e+02,1.000000,0.857000,1032.970000,5.861000,7.511000,19.535000,...,29.000000,5.440000,3.212000,10727.146000,233.070000,6.930000,0.000000,2.000000,74.160000,0.737000
75%,8.658650e+04,660.000000,665.286000,1.700250e+03,11.000000,11.429000,7132.510500,55.749000,61.828250,138.043250,...,38.000000,13.260000,8.353000,25063.846000,318.949000,9.750000,47.782000,3.600000,78.490000,0.828000
max,1.223157e+08,880902.000000,739564.429000,2.701445e+06,17895.000000,14424.000000,147945.383000,8652.658000,2648.773000,2327.774000,...,48.200000,27.049000,18.493000,116935.600000,724.417000,30.530000,98.999000,13.800000,86.750000,0.957000


In [5]:
ws = Workspace.from_config()
experiment_name = 'Covid19HyperDrExperiment'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=2, delay_evaluation=5)
from azureml.train.hyperdrive import GridParameterSampling
#TODO: Create the different params that you will be using during training
param_sampling = GridParameterSampling( 
    {
        '--C': choice(0.01, 0.1, 1, 10, 100), 
        '--max_iter': choice(25, 50, 100,150)
    }
)
if "HyperDrive_training" not in os.listdir():
    os.mkdir("./HyperDrive_training")
    os.mkdir("./HyperDrive_training/github")
training_folder = './HyperDrive_training/' 
kaggleDataset_folder = './HyperDrive_training/github/'   
os.makedirs(training_folder, exist_ok=True)
os.makedirs(kaggleDataset_folder, exist_ok=True)
shutil.copy('TrainCovid19Infections.py', training_folder)
shutil.copy('github/owid-covid-data.csv', kaggleDataset_folder)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=training_folder, compute_target= compute_target, entry_script='TrainCovid19Infections.py')

hyperdrive_run_config = HyperDriveConfig(
                                   hyperparameter_sampling = param_sampling,
                                   primary_metric_name = 'Accuracy',
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                   max_total_runs=100,
                                   max_concurrent_runs = 3,
                                   policy = early_termination_policy,
                                   estimator = estimator)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [7]:
#TODO: Submit your experiment
tag = {"Covid19Infections": "Capstone project: Covid19 HyperDrive Experiment"}
remote_run = experiment.submit(hyperdrive_run_config,tags=tag, show_output=True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_cee737cf-c6a2-44d7-81df-73945ef1e04c
Web View: https://ml.azure.com/experiments/Covid19HyperDrExperiment/runs/HD_cee737cf-c6a2-44d7-81df-73945ef1e04c?wsid=/subscriptions/c04b3d3f-4994-454d-96ff-aa3f2050b57f/resourcegroups/testingMLFunctionnalities/workspaces/OptimizePipeline

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-20T15:15:26.301889][API][INFO]Experiment created<END>\n""<START>[2021-03-20T15:15:27.386812][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-03-20T15:15:27.106817][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n"<START>[2021-03-20T15:15:27.5532878Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run_HyperDr = remote_run.get_best_run_by_primary_metric()

# Get the metrics of the bestselected run
best_run_metrics = best_run_HyperDr.get_metrics()
# Show the Accuracy of that run
print('Best Accuracy: {}'.format(best_run_metrics['Accuracy']))
best_run_HyperDr

In [ ]:
parameter_values = best_run_HyperDr.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run_HyperDr.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:',parameter_values[1])
print('\n max_iter:',parameter_values[3])


In [ ]:
#Save the best model
Hyp_DrCovid19_Model = best_run_HyperDr.register_model(model_name="HyperDrCovid19Model", model_path='outputs/model.joblib')
print(Hyp_DrCovid19_Model.name,": Version Number",Hyp_DrCovid19_Model.version, sep='\t')